In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

In [7]:
import os

data_folder = r"C:\Users\mrpap\OneDrive\Desktop\ecg signals\Raw dataset per subject ppg"

files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]


In [8]:
for file in files:
    file_path = os.path.join(data_folder, file)
    
    data = pd.read_csv(file_path)   # or read_csv(..., sep='\t') for txt
    
    signal = data['Red (a.u)']  # column name example
    
    # ---- process signal here ----
    print(f"Processing {file}")


Processing 1.csv
Processing 10.csv
Processing 11.csv
Processing 12.csv
Processing 13.csv
Processing 14.csv
Processing 15.csv
Processing 16.csv
Processing 17.csv
Processing 18.csv
Processing 19.csv
Processing 2.csv
Processing 20.csv
Processing 21.csv
Processing 22.csv
Processing 23.csv
Processing 24.csv
Processing 25.csv
Processing 26.csv
Processing 27.csv
Processing 28.csv
Processing 29.csv
Processing 3.csv
Processing 30.csv
Processing 31.csv
Processing 32.csv
Processing 33.csv
Processing 34.csv
Processing 35.csv
Processing 36.csv
Processing 37.csv
Processing 38.csv
Processing 39.csv
Processing 4.csv
Processing 40.csv
Processing 41.csv
Processing 42.csv
Processing 43.csv
Processing 44.csv
Processing 45.csv
Processing 46.csv
Processing 47.csv
Processing 48.csv
Processing 49.csv
Processing 5.csv
Processing 50.csv
Processing 51.csv
Processing 52.csv
Processing 53.csv
Processing 54.csv
Processing 55.csv
Processing 56.csv
Processing 57.csv
Processing 58.csv
Processing 59.csv
Processing 6.cs

In [12]:
#ppg_red = opt1['Red (a.u)'].value
#ppg_ir=opt1['Infra Red (a.u)'].value
fs = 25  # sampling frequency (Hz)
#signal = ppg_red
#t = np.arange(len(ppg_red)) / fs
N = 100
def demean(signal):
    """
    Remove DC component (mean) from a signal.
    
    Parameters:
        signal (array-like): Input signal
    
    Returns:
        demeaned_signal (np.ndarray)
        mean_value (float)
    """
    mean_value = np.mean(signal)
    demeaned_signal = signal - mean_value
    return demeaned_signal, mean_value


In [13]:
from scipy.signal import detrend as scipy_detrend

def detrend_signal(signal, detrend_type='linear'):
    """
    Remove trend from a signal.
    
    Parameters:
        signal (array-like): Input signal
        detrend_type (str): 'linear' or 'constant'
    
    Returns:
        detrended_signal (np.ndarray)
    """
    detrended_signal = scipy_detrend(signal, type=detrend_type)
    return detrended_signal


In [14]:
import numpy as np

def polynomial_detrend(signal, fs, degree=2):
    """
    Remove polynomial baseline trend from a signal.
    
    Parameters:
        signal (array-like): Input signal
        fs (float): Sampling frequency (Hz)
        degree (int): Polynomial degree (e.g., 1=linear, 2=quadratic)
    
    Returns:
        detrended_signal (np.ndarray)
        trend (np.ndarray)
        coeffs (np.ndarray)
    """
    signal = np.asarray(signal)
    t = np.arange(len(signal)) / fs
    
    coeffs = np.polyfit(t, signal, degree)
    trend = np.polyval(coeffs, t)
    
    detrended_signal = signal - trend
    return detrended_signal, trend, coeffs


In [15]:
from scipy.signal import butter, filtfilt
import numpy as np

def bandpass_filter(signal, fs, lowcut=0.5, highcut=4.0, order=3):
    """
    Apply Butterworth band-pass filter with zero-phase filtering.
    
    Parameters:
        signal (array-like): Input signal
        fs (float): Sampling frequency (Hz)
        lowcut (float): Low cutoff frequency (Hz)
        highcut (float): High cutoff frequency (Hz)
        order (int): Filter order
    
    Returns:
        filtered_signal (np.ndarray)
    """
    signal = np.asarray(signal)
    
    nyq = fs / 2
    low = lowcut / nyq
    high = highcut / nyq
    
    # Safety check
    if high >= 1 or low <= 0:
        raise ValueError("Cutoff frequencies must satisfy 0 < lowcut < highcut < fs/2")
    
    b, a = butter(order, [low, high], btype='bandpass')
    filtered_signal = filtfilt(b, a, signal)
    
    return filtered_signal


In [16]:
from scipy.signal import find_peaks
def peak_detector(signal, distance):
    min_distance = int(0.45 * fs)
    peaks, properties = find_peaks(
    signal, # might change after filters
    distance=min_distance
)
from scipy.signal import find_peaks
import numpy as np

def detect_ppg_peaks(signal, fs, min_rr=0.45, prominence=None, height=None):
    """
    Detect systolic peaks in a PPG signal.
    
    Parameters:
        signal (array-like): Filtered PPG signal
        fs (float): Sampling frequency (Hz)
        min_rr (float): Minimum RR interval in seconds (controls max HR)
        prominence (float or None): Peak prominence constraint
        height (float or None): Minimum peak height
    
    Returns:
        peaks (np.ndarray): Indices of detected peaks
        properties (dict): Peak properties from scipy
    """
    signal = np.asarray(signal)
    
    min_distance = int(min_rr * fs)
    
    peaks, properties = find_peaks(
        signal,
        distance=min_distance,
        prominence=prominence,
        height=height
    )
    
    return peaks, properties


    
    

In [17]:
import numpy as np

def refine_peaks(signal, peaks, fs, search_window=0.1):
    """
    Refine peak locations by searching for the true local maximum.
    
    Parameters:
        signal (array-like): Filtered signal
        peaks (array-like): Initial peak indices
        fs (float): Sampling frequency (Hz)
        search_window (float): Half window size in seconds
    
    Returns:
        refined_peaks (np.ndarray): Refined peak indices
    """
    signal = np.asarray(signal)
    peaks = np.asarray(peaks)
    
    window = int(search_window * fs)
    refined_peaks = []

    for p in peaks:
        start = max(p - window, 0)
        end   = min(p + window, len(signal))
        local_max = np.argmax(signal[start:end]) + start
        refined_peaks.append(local_max)

    return np.array(refined_peaks)


In [18]:
import numpy as np

def detect_ppg_valleys(signal, peaks, fs, search_window=0.6, min_segment_len=6):
    """
    Detect pulse foot (valley) locations using max-slope method.
    
    Parameters:
        signal (array-like): Filtered PPG signal
        peaks (array-like): Refined systolic peak indices
        fs (float): Sampling frequency (Hz)
        search_window (float): Time (s) before peak to search for valley
        min_segment_len (int): Minimum samples needed to process a segment
    
    Returns:
        valleys (np.ndarray): Valley indices (NaN if not detected)
    """
    signal = np.asarray(signal)
    peaks = np.asarray(peaks)
    
    window = int(search_window * fs)
    valleys = []

    for p in peaks:
        start = max(p - window, 0)
        segment = signal[start:p]

        if len(segment) < min_segment_len:
            valleys.append(np.nan)
            continue

        # First derivative
        d1 = np.diff(segment)

        # Index of maximum upstroke
        max_slope_idx = np.argmax(d1)

        # Search minimum BEFORE max slope
        foot_segment = segment[:max_slope_idx + 1]
        valley_idx = np.argmin(foot_segment)

        valleys.append(start + valley_idx)

    return np.array(valleys)


In [19]:
import numpy as np

def detect_dicrotic_notches(signal, valleys, peaks, min_segment_len=6):
    """
    Detect dicrotic notches between pulse foot (valley) and systolic peak.
    
    Parameters:
        signal (array-like): PPG signal (band-pass or low-pass filtered)
        valleys (array-like): Valley (pulse foot) indices
        peaks (array-like): Refined systolic peak indices
        min_segment_len (int): Minimum segment length to analyze
    
    Returns:
        dicrotic_notches (np.ndarray): Indices of detected notches (NaN if not found)
    """
    signal = np.asarray(signal)
    valleys = np.asarray(valleys)
    peaks = np.asarray(peaks)
    
    dicrotic_notches = []

    for v, p in zip(valleys, peaks):

        # Skip invalid beats
        if np.isnan(v) or np.isnan(p):
            dicrotic_notches.append(np.nan)
            continue

        v = int(v)
        p = int(p)

        if p <= v:
            dicrotic_notches.append(np.nan)
            continue

        segment = signal[v:p]

        if len(segment) < min_segment_len:
            dicrotic_notches.append(np.nan)
            continue

        # First derivative
        d1 = np.diff(segment)

        # Zero-crossing: negative → positive (local minimum)
        candidates = np.where((d1[:-1] < 0) & (d1[1:] > 0))[0]

        if len(candidates) == 0:
            dicrotic_notches.append(np.nan)
        else:
            notch_idx = v + candidates[0]
            dicrotic_notches.append(notch_idx)

    return np.array(dicrotic_notches)


In [20]:
import numpy as np

def refine_valid_beats(peaks, valleys, notches):
    """
    Keep only beats where peak, valley, and dicrotic notch are all valid.
    
    Parameters:
        peaks (array-like): Refined peak indices
        valleys (array-like): Valley indices
        notches (array-like): Dicrotic notch indices
    
    Returns:
        peaks_v (np.ndarray): Valid peak indices
        valleys_v (np.ndarray): Valid valley indices
        notches_v (np.ndarray): Valid notch indices
        valid_mask (np.ndarray): Boolean mask of valid beats
    """
    peaks = np.asarray(peaks)
    valleys = np.asarray(valleys)
    notches = np.asarray(notches)
    
    valid_mask = (
        ~np.isnan(peaks) &
        ~np.isnan(valleys) &
        ~np.isnan(notches)
    )
    
    peaks_v   = peaks[valid_mask].astype(int)
    valleys_v = valleys[valid_mask].astype(int)
    notches_v = notches[valid_mask].astype(int)
    
    return peaks_v, valleys_v, notches_v, valid_mask


In [21]:
import numpy as np

def extract_time_domain_features(signal, peaks, valleys, notches, fs):
    """
    Extract time-domain PPG pulse features.
    
    Parameters:
        signal (array-like): PPG signal
        peaks (array-like): Valid peak indices
        valleys (array-like): Valid valley indices
        notches (array-like): Valid dicrotic notch indices
        fs (float): Sampling frequency (Hz)
    
    Returns:
        features (dict): Dictionary of feature arrays
    """
    signal = np.asarray(signal)

    # Pulse amplitude
    pulse_amplitude = signal[peaks] - signal[valleys]

    # Timing features
    rise_time = (peaks - valleys) / fs
    notch_time = (notches - valleys) / fs
    peak_notch_delay = (peaks - notches) / fs

    # Area under pulse
    AUP = []
    for v, p in zip(valleys, peaks):
        area = np.trapezoid(signal[v:p])
        AUP.append(abs(area))
    AUP = np.array(AUP)

    # Reflection Index
    RI = (
        (signal[notches] - signal[valleys]) /
        (signal[peaks]   - signal[valleys])
    )

    # Pulse width at half amplitude
    pulse_width = []
    for v, p in zip(valleys, peaks):
        half_amp = signal[v] + 0.5 * (signal[p] - signal[v])
        segment = signal[v:p]
        idx = np.where(segment >= half_amp)[0]
        if len(idx) > 0:
            pulse_width.append(idx[-1] / fs)
        else:
            pulse_width.append(np.nan)
    pulse_width = np.array(pulse_width)

    # Inter-beat interval and heart rate
    IBI = np.diff(peaks) / fs
    HR  = 60 / IBI

    return {
        "pulse_amplitude": pulse_amplitude,
        "rise_time": rise_time,
        "notch_time": notch_time,
        "peak_notch_delay": peak_notch_delay,
        "AUP": AUP,
        "RI": RI,
        "pulse_width": pulse_width,
        "IBI": IBI,
        "HR": HR
    }


In [22]:
import numpy as np

def extract_frequency_domain_features(signal, fs, max_freq=5.0):
    """
    Extract frequency-domain features from PPG signal.
    
    Parameters:
        signal (array-like): Preprocessed PPG signal
        fs (float): Sampling frequency (Hz)
        max_freq (float): Maximum frequency to analyze (Hz)
    
    Returns:
        features (dict)
        freqs (np.ndarray)
        spectrum (np.ndarray)
    """
    signal = np.asarray(signal)
    N = len(signal)

    # FFT
    fft_vals = np.fft.fft(signal)
    freqs = np.fft.fftfreq(N, d=1/fs)

    # Positive frequencies
    pos_mask = freqs >= 0
    freqs_pos = freqs[pos_mask]
    fft_mag = np.abs(fft_vals[pos_mask])

    # Limit to physiological band
    band_mask = freqs_pos <= max_freq
    freqs_band = freqs_pos[band_mask]
    mag_band = fft_mag[band_mask]

    # ---- Frequency-domain features ----
    dominant_freq = freqs_band[np.argmax(mag_band)]  # Hz
    spectral_centroid = np.sum(freqs_band * mag_band) / np.sum(mag_band)
    total_power = np.sum(mag_band**2)

    return {
        "dominant_freq": dominant_freq,
        "spectral_centroid": spectral_centroid,
        "total_power": total_power
    }, freqs_band, mag_band


In [23]:
from scipy.signal import welch
import numpy as np

def compute_welch_psd(
    signal,
    fs,
    window='hann',
    nperseg=256,
    noverlap=None,
    scaling='density'
):
    """
    Compute Power Spectral Density using Welch's method.
    
    Parameters:
        signal (array-like): Input signal
        fs (float): Sampling frequency (Hz)
        window (str): Window type
        nperseg (int): Length of each segment
        noverlap (int or None): Overlap between segments
        scaling (str): 'density' or 'spectrum'
    
    Returns:
        f_welch (np.ndarray): Frequency axis (Hz)
        psd (np.ndarray): Power spectral density
    """
    signal = np.asarray(signal)
    
    # Safety: nperseg should not exceed signal length
    nperseg = min(nperseg, len(signal))
    
    f_welch, psd = welch(
        signal,
        fs=fs,
        window=window,
        nperseg=nperseg,
        noverlap=noverlap,
        scaling=scaling
    )
    
    return f_welch, psd


In [24]:
import numpy as np

def extract_welch_frequency_features(f_welch, psd, cardiac_band=(0.5, 4.0)):
    """
    Extract frequency-domain features from Welch PSD.
    
    Parameters:
        f_welch (np.ndarray): Frequency axis
        psd (np.ndarray): Power spectral density
        cardiac_band (tuple): (low, high) Hz
    
    Returns:
        features (dict)
    """
    # Dominant frequency
    idx_peak = np.argmax(psd)
    dominant_freq = f_welch[idx_peak]
    HR_spectral = dominant_freq * 60

    # Cardiac band power
    band_mask = (f_welch >= cardiac_band[0]) & (f_welch <= cardiac_band[1])
    cardiac_power = np.trapezoid(psd[band_mask], f_welch[band_mask])

    # Total power
    total_power = np.trapezoid(psd, f_welch)
    norm_cardiac_power = cardiac_power / total_power

    # Spectral centroid
    spectral_centroid = np.sum(f_welch * psd) / np.sum(psd)

    # Spectral bandwidth
    spectral_bandwidth = np.sqrt(
        np.sum(((f_welch - spectral_centroid)**2) * psd) / np.sum(psd)
    )

    # Spectral entropy
    psd_norm = psd / np.sum(psd)
    spectral_entropy = -np.sum(psd_norm * np.log2(psd_norm + 1e-12))

    return {
        "dominant_freq": dominant_freq,
        "HR_spectral": HR_spectral,
        "cardiac_power": cardiac_power,
        "normalized_cardiac_power": norm_cardiac_power,
        "spectral_centroid": spectral_centroid,
        "spectral_bandwidth": spectral_bandwidth,
        "spectral_entropy": spectral_entropy
    }


In [25]:
import numpy as np

def summarize_subject_features(
    pulse_amplitude,
    rise_time,
    notch_time,
    peak_notch_delay,
    RI,
    pulse_width,
    AUP,
    HR,
    dominant_freq,
    HR_spectral,
    cardiac_power,
    norm_cardiac_power,
    spectral_centroid,
    spectral_bandwidth,
    spectral_entropy
):
    """
    Aggregate beat-wise and spectral features into subject-level features.
    
    Returns:
        features (dict): One-row feature dictionary
    """
    return {
        # ---- Time-domain summaries ----
        "mean_pulse_amplitude": np.nanmean(pulse_amplitude),
        "std_pulse_amplitude":  np.nanstd(pulse_amplitude),

        "mean_rise_time_s": np.nanmean(rise_time),
        "mean_notch_time_s": np.nanmean(notch_time),
        "mean_peak_notch_delay_s": np.nanmean(peak_notch_delay),

        "mean_RI": np.nanmean(RI),

        "mean_pulse_width_s": np.nanmean(pulse_width),
        "mean_AUP": np.nanmean(AUP),

        "mean_HR_bpm": np.nanmean(HR),
        "HR_std": np.nanstd(HR),

        # ---- Frequency-domain features ----
        "dominant_frequency_hz": dominant_freq,
        "spectral_HR_bpm": HR_spectral,

        "cardiac_band_power": cardiac_power,
        "normalized_cardiac_power": norm_cardiac_power,

        "spectral_centroid_hz": spectral_centroid,
        "spectral_bandwidth_hz": spectral_bandwidth,

        "spectral_entropy": spectral_entropy
    }


In [26]:
import os
import pandas as pd

data_folder =  r"C:\Users\mrpap\OneDrive\Desktop\ecg signals\Raw dataset per subject ppg"  # folder with PPG files
fs = 25                # sampling frequency

all_subjects = []

for file in os.listdir(data_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(data_folder, file)

    # 1. Load signal
    data = pd.read_csv(file_path)
    ppg = data['Red (a.u)'].values

    #ppg = data.iloc[:, 0].values

    # 2. Preprocessing
    ppg, _ = demean(ppg)
    ppg, _, _ = polynomial_detrend(ppg, fs, degree=2)
    ppg = bandpass_filter(ppg, fs, 0.5, 4.0)

    # 3. Landmark detection
    peaks, _ = detect_ppg_peaks(ppg, fs)
    peaks = refine_peaks(ppg, peaks, fs)
    valleys = detect_ppg_valleys(ppg, peaks, fs)
    notches = detect_dicrotic_notches(ppg, valleys, peaks)

    # 4. Beat validation
    peaks_v, valleys_v, notches_v, _ = refine_valid_beats(
        peaks, valleys, notches
    )

    if len(peaks_v) < 3:
        continue  # skip bad subjects

    # 5. Time-domain features
    time_feats = extract_time_domain_features(
        ppg, peaks_v, valleys_v, notches_v, fs
    )

    # 6. Frequency-domain (Welch)
    f_welch, psd = compute_welch_psd(ppg, fs)
    freq_feats = extract_welch_frequency_features(f_welch, psd)

    # 7. Subject-level summary
    subject_features = summarize_subject_features(
        time_feats["pulse_amplitude"],
        time_feats["rise_time"],
        time_feats["notch_time"],
        time_feats["peak_notch_delay"],
        time_feats["RI"],
        time_feats["pulse_width"],
        time_feats["AUP"],
        time_feats["HR"],
        freq_feats["dominant_freq"],
        freq_feats["HR_spectral"],
        freq_feats["cardiac_power"],
        freq_feats["normalized_cardiac_power"],
        freq_feats["spectral_centroid"],
        freq_feats["spectral_bandwidth"],
        freq_feats["spectral_entropy"]
    )

    subject_features["subject_id"] = file
    all_subjects.append(subject_features)


In [29]:
df = pd.DataFrame(all_subjects)
df.to_excel("PPG_Features.xlsx", index=False)
import os
print(os.getcwd())


C:\Users\mrpap
